In [1]:
from model import Deeplabv3

deeplab_model = Deeplabv3(backbone="xception", input_shape=(512, 512, 3), classes=19, weights='cityscapes')

Cityscapes!


In [2]:
deeplab_model.summary()

Model: "deeplabv3plus"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         lambda[0][0]                     
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
______________________________________________________________________________________

In [3]:
import pandas as pd
from PIL import Image

dataset = pd.read_csv('../../kitti360_dataset_truncated.csv')
dataset = dataset[dataset['subset'] == 'test']
images = dataset.x.tolist()
labels = dataset.y.tolist()

In [4]:
# Generates labels using most basic setup.  Supports various image sizes.  Returns image labels in same format
# as original image.  Normalization matches MobileNetV2

import cv2
import numpy as np

trained_image_width=512 
mean_subtraction_value=127.5
image = cv2.imread(images[5], 1)

# resize to max dimension of images from training dataset
w, h, _ = image.shape
ratio = float(trained_image_width) / np.max([w, h])
resized_image = np.array(Image.fromarray(image.astype('uint8')).resize((int(ratio * h), int(ratio * w))))

# apply normalization for trained dataset images
resized_image = (resized_image / mean_subtraction_value) - 1.

# pad array to square image to match training images
pad_x = int(trained_image_width - resized_image.shape[0])
pad_y = int(trained_image_width - resized_image.shape[1])
resized_image = np.pad(resized_image, ((0, pad_x), (0, pad_y), (0, 0)), mode='constant')

# make prediction
res = deeplab_model.predict(np.expand_dims(resized_image, 0))
preds_lab = np.argmax(res.squeeze(), -1)
preds_lab_down = np.argmax(res.squeeze(), -1)

#remove padding and resize back to original image
if pad_x > 0:
    preds_lab = preds_lab[:-pad_x]
if pad_y > 0:
    preds_lab = preds_lab[:, :-pad_y]
preds_lab = np.array(Image.fromarray(preds_lab.astype('uint8')).resize((h, w))) 
preds_lab_down = np.array(Image.fromarray(preds_lab_down.astype('uint8')).resize((h, w))) 

NotFoundError:  No algorithm worked!
	 [[node deeplabv3plus/entry_flow_conv1_1/Conv2D (defined at <ipython-input-4-5a040729029c>:25) ]] [Op:__inference_predict_function_11531]

Function call stack:
predict_function
